In [1]:
import os
import os.path

import cv2
import torch
import torchvision

/home/binhnq/anaconda3/envs/object_tracking/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
video_path = os.path.join('videos/Traffic.mp4')

stream = "video"

video = torchvision.io.VideoReader(video_path, stream)
print(video.get_metadata())

Using cache found in /home/binhnq/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2022-10-6 Python-3.10.6 torch-1.12.1 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


{'video': {'duration': [11.177833], 'fps': [29.97002997002997]}, 'audio': {'duration': [], 'framerate': []}, 'subtitles': {'duration': []}, 'cc': {'duration': []}}


In [100]:
from deep_sort_realtime.deepsort_tracker import DeepSort

tracker = DeepSort(max_age=3)

MobileNetV2 Embedder for Deep Sort initialised
- gpu enabled: False
- half precision: False
- max batch size: 16
- expects BGR: True
DeepSort Tracker initialised
- max age: 1
- appearance threshold: 0.2
- nms threshold: OFF
- max num of appearance features: None
- overriding track class : No
- today given : No
- in-build embedder : Yes
- polygon detections : No


In [107]:
frame = next(video)

In [108]:
results = model([frame['data'].numpy()])

In [115]:
xywh

tensor([[1.04094e+03, 1.41036e+02, 5.13522e+01, 4.92366e+01, 8.18564e-01, 2.00000e+00],
        [1.11670e+03, 1.38392e+02, 5.58112e+01, 4.71433e+01, 8.12836e-01, 2.00000e+00],
        [1.01192e+03, 8.51707e+01, 3.80070e+01, 2.84766e+01, 6.14487e-01, 2.00000e+00],
        [1.76400e+03, 7.56552e+01, 6.10818e+01, 3.22389e+01, 5.71312e-01, 2.00000e+00],
        [1.69240e+03, 7.24416e+01, 5.30519e+01, 3.39321e+01, 4.80228e-01, 2.00000e+00],
        [1.07699e+03, 1.41601e+02, 1.27404e+02, 5.04145e+01, 4.51903e-01, 2.00000e+00],
        [9.82593e+02, 5.24337e+01, 3.17847e+01, 2.57322e+01, 3.71623e-01, 2.00000e+00],
        [1.03509e+03, 4.92508e+01, 2.22184e+01, 2.21722e+01, 2.85891e-01, 2.00000e+00]])

In [109]:
xywh = results.xywh[0]

list_xywh = xywh[xywh[:, -1] == 2].tolist()

detections = []
for xywh_item in list_xywh:
    detections.append((xywh_item[:4], xywh_item[4], results.names[xywh_item[5]]))
tracks = tracker.update_tracks(detections, frame=results.ims[0])

for track in tracks:
    if not track.is_confirmed():
        continue
    track_id = track.track_id
    ltrb = track.to_ltrb()
    print(track_id)
    print(ltrb)
    img

1
[     1040.5      140.56      1091.7      189.26]
2
[     1117.1      137.96      1171.1       185.2]
3
[     1766.3      76.002      1825.6      108.12]
4
[     1011.7      84.917      1049.7      113.35]
5
[     1692.2      73.268      1750.6      107.35]
6
[     1079.2      141.14      1201.3      191.55]
7
[     983.85      52.335      1012.6      78.236]


In [119]:
??track

Type:        Track
String form: <deep_sort_realtime.deep_sort.track.Track object at 0x7fb572763f70>
File:        ~/anaconda3/envs/object_tracking/lib/python3.10/site-packages/deep_sort_realtime/deep_sort/track.py
Source:     
class Track:
    """
    A single target track with state space `(x, y, a, h)` and associated
    velocities, where `(x, y)` is the center of the bounding box, `a` is the
    aspect ratio and `h` is the height.

    Parameters
    ----------
    mean : ndarray
        Mean vector of the initial state distribution.
    covariance : ndarray
        Covariance matrix of the initial state distribution.
    track_id : int
        A unique track identifier.
    n_init : int
        Number of consecutive detections before the track is confirmed. The
        track state is set to `Deleted` if a miss occurs within the first
        `n_init` frames.
    max_age : int
        The maximum number of consecutive misses before the track state is
        set to `Deleted`.
    fea

In [114]:
def track_label(img, track_id, ltrb):
    thick_box = 1
    color = [0, 255, 0]
    cv2.rectangle(img_vis, (ltrb[0], ltrb[1]), (ltrb[2], ltrb[3]), color_box, thick_box)
    cv2.putText(img_vis, "{}".format(track_id), (ltrb[0], ltrb[1]), cv2.FONT_HERSHEY_DUPLEX, 0.4,
                color, 1)
    return img

In [120]:
video.get_metadata()

{'video': {'duration': [11.177833], 'fps': [29.97002997002997]},
 'audio': {'duration': [], 'framerate': []},
 'subtitles': {'duration': []},
 'cc': {'duration': []}}

In [121]:
??video

Type:        VideoReader
String form: <torchvision.io.video_reader.VideoReader object at 0x7fb5feb635b0>
File:        ~/anaconda3/envs/object_tracking/lib/python3.10/site-packages/torchvision/io/video_reader.py
Source:     
class VideoReader:
    """
    Fine-grained video-reading API.
    Supports frame-by-frame reading of various streams from a single video
    container.

    .. betastatus:: VideoReader class

    Example:
        The following examples creates a :mod:`VideoReader` object, seeks into 2s
        point, and returns a single frame::

            import torchvision
            video_path = "path_to_a_test_video"
            reader = torchvision.io.VideoReader(video_path, "video")
            reader.seek(2.0)
            frame = next(reader)

        :mod:`VideoReader` implements the iterable API, which makes it suitable to
        using it in conjunction with :mod:`itertools` for more advanced reading.
        As such, we can use a :mod:`VideoReader` instance inside for